<a href="https://colab.research.google.com/github/Soy-code/Code-Up/blob/master/%EA%B8%B0%EC%B6%9C%EB%AC%B8%EC%A0%9C_TF3_rps_(%EB%AC%B8%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this task you will build a classifier for Rock-Paper-Scissors 
# based on the rps dataset.
#
# IMPORTANT: Your final layer should be as shown, do not change the
# provided code, or the tests may fail
#
# IMPORTANT: Images will be tested as 150x150 with 3 bytes of color depth
# So ensure that your input layer is designed accordingly, or the tests
# may fail. 
#
# NOTE THAT THIS IS UNLABELLED DATA. 
# You can use the ImageDataGenerator to automatically label it
# and we have provided some starter code.

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 3 - rps
# val_loss: 0.0871
# val_acc: 0.97
# =================================================== #
# =================================================== #

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    urllib.request.urlretrieve(url, 'rps.zip')
    local_zip = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/')
    zip_ref.close()

    TRAINING_DIR = "tmp/rps/"
    training_datagen = ImageDataGenerator(
            rescale =  1. / 255,   # 픽셀값 조절
            rotation_range = 40,    # 이미지 회전
            width_shift_range = 0.2,   # 가로 방향 이동
            height_shift_range = 0.2,    # 세로 방향 이동
            shear_range = 0.2,   # 이미지 굴절
            zoom_range = 0.2,    # 이미지 확대
            horizontal_flip = True,    # 횡 방향으로 이미지 반전
            fill_mode = 'nearest',   # 빈 픽셀값에 대하여 값을 채우는 방식
            validation_split = 0.2    # --> train데이터 valid데이터 split
    )

    train_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size = 32, 
        target_size = (150, 150),
        class_mode = 'categorical',   # 출력층이 sigmoid인 경우에는 binary로 넣어줘야 함 
        subset = 'training',
    )

    validation_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size = 32, 
        target_size = (150, 150), 
        class_mode = 'categorical', 
        subset = 'validation',   
    )


    model = tf.keras.models.Sequential([
            Conv2D(128, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
            MaxPooling2D(2, 2), 
            Conv2D(64, (3, 3), activation = 'relu'), 
            MaxPooling2D(2, 2),
            Conv2D(128, (3, 3), activation = 'relu'), 
            MaxPooling2D(2, 2), 
            Conv2D(64, (3, 3), activation = 'relu'), 
            MaxPooling2D(2, 2),
            Flatten(), 
            Dropout(0.3), 
            Dense(128, activation = 'relu'), 
            Dropout(0.3),
            Dense(64, activation = 'relu'), 
    # YOUR CODE HERE, BUT END WITH A 3 Neuron Dense, activated by softmax
            tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

    # 체크포인트 생성
    checkpoint_path = 'my_checkpoint_3-rps-1.ckpt'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, 
                                 save_best_only = True, 
                                 save_weights_only = True, 
                                 monitor = 'val_loss', 
                                 verbose = 1)

    # 모델 학습
    model.fit(train_generator, 
              validation_data = (validation_generator), 
              epochs = 15, 
              callbacks = [checkpoint])
    
    # best 모델 로드드
    model.load_weights(checkpoint_path)

    # 모델 검증
    model.evaluate(validation_generator)  # [0.17 0.93]
    
    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-rps-1.h5")


Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Epoch 1/15
63/63 [==============================] - ETA: 0s - loss: 1.1009 - acc: 0.3194
Epoch 1: val_loss improved from inf to 1.11683, saving model to my_checkpoint_3-rps-1.ckpt
63/63 [==============================] - 21s 327ms/step - loss: 1.1009 - acc: 0.3194 - val_loss: 1.1168 - val_acc: 0.3333
Epoch 2/15
63/63 [==============================] - ETA: 0s - loss: 1.0802 - acc: 0.3993
Epoch 2: val_loss improved from 1.11683 to 1.08473, saving model to my_checkpoint_3-rps-1.ckpt
63/63 [==============================] - 20s 311ms/step - loss: 1.0802 - acc: 0.3993 - val_loss: 1.0847 - val_acc: 0.3274
Epoch 3/15
63/63 [==============================] - ETA: 0s - loss: 0.9133 - acc: 0.5957
Epoch 3: val_loss improved from 1.08473 to 0.93346, saving model to my_checkpoint_3-rps-1.ckpt
63/63 [==============================] - 20s 311ms/step - loss: 0.9133 - acc: 0.5957 - val_loss: 0.9335 - val_acc: 0.4980
Epo

In [5]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this task you will build a classifier for Rock-Paper-Scissors 
# based on the rps dataset.
#
# IMPORTANT: Your final layer should be as shown, do not change the
# provided code, or the tests may fail
#
# IMPORTANT: Images will be tested as 150x150 with 3 bytes of color depth
# So ensure that your input layer is designed accordingly, or the tests
# may fail. 
#
# NOTE THAT THIS IS UNLABELLED DATA. 
# You can use the ImageDataGenerator to automatically label it
# and we have provided some starter code.

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 3 - rps
# val_loss: 0.0871
# val_acc: 0.97
# =================================================== #
# =================================================== #

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    urllib.request.urlretrieve(url, 'rps.zip')
    local_zip = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/')
    zip_ref.close()

    TRAINING_DIR = "tmp/rps/"
    training_datagen = ImageDataGenerator(
        rescale =  1. / 255,   # 픽셀값 조절
        rotation_range = 30,    # 이미지 회전
        width_shift_range = 0.15,   # 가로 방향 이동
        height_shift_range = 0.15,    # 세로 방향 이동
        shear_range = 0.2,   # 이미지 굴절
        zoom_range = 0.2,    # 이미지 확대
        horizontal_flip = True,    # 횡 방향으로 이미지 반전
        fill_mode = 'nearest',   # 빈 픽셀값에 대하여 값을 채우는 방식
        validation_split = 0.2    # --> train데이터 valid데이터 split
    )

    train_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size = 32, 
        target_size = (150, 150),
        class_mode = 'categorical',   # 출력층이 sigmoid인 경우에는 binary로 넣어줘야 함 
        subset = 'training',
    )

    validation_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size = 32, 
        target_size = (150, 150), 
        class_mode = 'categorical', 
        subset = 'validation',   
    )

    # 모델 정의
    model = Sequential([
        Conv2D(64, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
        MaxPooling2D(2, 2), 
        Conv2D(64, (3, 3), activation = 'relu'), 
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation = 'relu'), 
        MaxPooling2D(2, 2), 
        Conv2D(128, (3, 3), activation = 'relu'), 
        MaxPooling2D(2, 2),
        Flatten(), 
        Dropout(0.5), 
        Dense(512, activation = 'relu'), 
        Dense(3, activation = 'softmax')
    ])

    # 모델 생성
    # IDG는 자동으로 one hot encoding 해줌. 
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
    # optimizer = 'rmsprop'

    # 체크포인트 생성
    checkpoint_path = 'my_checkpoint_3-rps-2.ckpt'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, 
                                 save_best_only = True, 
                                 save_weights_only = True, 
                                 monitor = 'val_loss', 
                                 verbose = 1)

    # 모델 학습
    model.fit(train_generator, 
          validation_data = (validation_generator), 
          epochs = 20, 
          callbacks = [checkpoint])

    # best 모델 로드드
    model.load_weights(checkpoint_path)

    # 모델 검증
    print(model.evaluate(validation_generator))  # [0.16, 0.94]
    
    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-rps-2.h5")


Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Epoch 1/20
63/63 [==============================] - ETA: 0s - loss: 1.0405 - acc: 0.4658
Epoch 1: val_loss improved from inf to 1.03849, saving model to my_checkpoint_3-rps-2.ckpt
63/63 [==============================] - 21s 315ms/step - loss: 1.0405 - acc: 0.4658 - val_loss: 1.0385 - val_acc: 0.4464
Epoch 2/20
63/63 [==============================] - ETA: 0s - loss: 0.6898 - acc: 0.7183
Epoch 2: val_loss improved from 1.03849 to 0.76584, saving model to my_checkpoint_3-rps-2.ckpt
63/63 [==============================] - 19s 306ms/step - loss: 0.6898 - acc: 0.7183 - val_loss: 0.7658 - val_acc: 0.6687
Epoch 3/20
63/63 [==============================] - ETA: 0s - loss: 0.3375 - acc: 0.8735
Epoch 3: val_loss improved from 0.76584 to 0.50032, saving model to my_checkpoint_3-rps-2.ckpt
63/63 [==============================] - 19s 301ms/step - loss: 0.3375 - acc: 0.8735 - val_loss: 0.5003 - val_acc: 0.8294
Epo

In [6]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this task you will build a classifier for Rock-Paper-Scissors 
# based on the rps dataset.
#
# IMPORTANT: Your final layer should be as shown, do not change the
# provided code, or the tests may fail
#
# IMPORTANT: Images will be tested as 150x150 with 3 bytes of color depth
# So ensure that your input layer is designed accordingly, or the tests
# may fail. 
#
# NOTE THAT THIS IS UNLABELLED DATA. 
# You can use the ImageDataGenerator to automatically label it
# and we have provided some starter code.

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 3 - rps
# val_loss: 0.0871
# val_acc: 0.97
# =================================================== #
# =================================================== #

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    urllib.request.urlretrieve(url, 'rps.zip')
    local_zip = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/')
    zip_ref.close()

    TRAINING_DIR = "tmp/rps/"
    training_datagen = ImageDataGenerator(
        rescale =  1. / 255,   # 픽셀값 조절
        rotation_range = 40,    # 이미지 회전
        width_shift_range = 0.2,   # 가로 방향 이동
        height_shift_range = 0.2,    # 세로 방향 이동
        shear_range = 0.2,   # 이미지 굴절
        zoom_range = 0.2,    # 이미지 확대
        horizontal_flip = True,    # 횡 방향으로 이미지 반전
        fill_mode = 'nearest',   # 빈 픽셀값에 대하여 값을 채우는 방식
        validation_split = 0.2    # --> train데이터 valid데이터 split
    )

    train_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size = 32, 
        target_size = (150, 150),
        class_mode = 'categorical',   # 출력층이 sigmoid인 경우에는 binary로 넣어줘야 함 
        subset = 'training',
    )

    validation_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size = 32, 
        target_size = (150, 150), 
        class_mode = 'categorical', 
        subset = 'validation',   
    )

    # 모델 정의
    model = Sequential([
        Conv2D(128, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
        MaxPooling2D(2, 2), 
        Conv2D(128, (3, 3), activation = 'relu'), 
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation = 'relu'), 
        MaxPooling2D(2, 2), 
        Conv2D(32, (3, 3), activation = 'relu'), 
        MaxPooling2D(2, 2),
        Flatten(), 
        Dropout(0.5), 
        Dense(512, activation = 'relu'), 
        Dropout(0.2), 
        Dense(128, activation = 'relu'), 
        Dense(32, activation = 'relu'), 
        Dense(3, activation = 'softmax')
    ])

    # 모델 생성
    # IDG는 자동으로 one hot encoding 해줌. 
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
    # optimizer = 'rmsprop'

    # 체크포인트 생성
    checkpoint_path = 'my_checkpoint_3-rps-3.ckpt'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, 
                                 save_best_only = True, 
                                 save_weights_only = True, 
                                 monitor = 'val_loss', 
                                 verbose = 1)

    # 모델 학습
    model.fit(train_generator, 
          validation_data = (validation_generator), 
          epochs = 20, 
          callbacks = [checkpoint])

    # best 모델 로드드
    model.load_weights(checkpoint_path)

    # 모델 검증
    print(model.evaluate(validation_generator))  # [0.16, 0.94]
    
    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-rps-3.h5")


Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Epoch 1/20
63/63 [==============================] - ETA: 0s - loss: 1.1023 - acc: 0.3512
Epoch 1: val_loss improved from inf to 1.09760, saving model to my_checkpoint_3-rps-3.ckpt
63/63 [==============================] - 22s 322ms/step - loss: 1.1023 - acc: 0.3512 - val_loss: 1.0976 - val_acc: 0.3333
Epoch 2/20
63/63 [==============================] - ETA: 0s - loss: 1.0793 - acc: 0.3919
Epoch 2: val_loss improved from 1.09760 to 1.01717, saving model to my_checkpoint_3-rps-3.ckpt
63/63 [==============================] - 20s 314ms/step - loss: 1.0793 - acc: 0.3919 - val_loss: 1.0172 - val_acc: 0.5278
Epoch 3/20
63/63 [==============================] - ETA: 0s - loss: 0.7577 - acc: 0.6553
Epoch 3: val_loss improved from 1.01717 to 0.85929, saving model to my_checkpoint_3-rps-3.ckpt
63/63 [==============================] - 20s 320ms/step - loss: 0.7577 - acc: 0.6553 - val_loss: 0.8593 - val_acc: 0.5298
Epo